In [20]:
import os
from glob import glob 

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow.keras.backend as K

%matplotlib inline

In [21]:
# 더 적은 쪽을 기준으로 가져온다 
tmp_label_paths = glob('../day1/ADE20K_2017_05_30_consistency/images/consistencyanalysis/original_ade20k/*_seg.png')

image_paths = []
label_paths = []

for lbl_path in tmp_label_paths:
    img_path = lbl_path.replace('_seg.png', '.jpg')
    if os.path.exists(img_path):
        image_paths.append(img_path)
        label_paths.append(lbl_path)
        
len(image_paths), len(label_paths)

(64, 64)

In [22]:
class_nums = np.arange(13)
class_nums

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

# Hyperparameter

In [23]:
batch_size = 4
epochs = 5
drop_rate = 0.2

input_shape = (128, 128, 3)

In [24]:
label_paths

['../day1/ADE20K_2017_05_30_consistency/images/consistencyanalysis/original_ade20k\\labelme_bxvctltntrtinjf_seg.png',
 '../day1/ADE20K_2017_05_30_consistency/images/consistencyanalysis/original_ade20k\\labelme_dfmtscjlgpxnxvg_seg.png',
 '../day1/ADE20K_2017_05_30_consistency/images/consistencyanalysis/original_ade20k\\labelme_dvkvazcsyvkumig_seg.png',
 '../day1/ADE20K_2017_05_30_consistency/images/consistencyanalysis/original_ade20k\\labelme_fbpgpaxesdmtnqy_seg.png',
 '../day1/ADE20K_2017_05_30_consistency/images/consistencyanalysis/original_ade20k\\labelme_hueaxalkegppjob_seg.png',
 '../day1/ADE20K_2017_05_30_consistency/images/consistencyanalysis/original_ade20k\\labelme_ksbcdzlsogcuudh_seg.png',
 '../day1/ADE20K_2017_05_30_consistency/images/consistencyanalysis/original_ade20k\\labelme_kvyxvreglagziop_seg.png',
 '../day1/ADE20K_2017_05_30_consistency/images/consistencyanalysis/original_ade20k\\labelme_nxqdbafndbozeee_seg.png',
 '../day1/ADE20K_2017_05_30_consistency/images/consisten

In [19]:
label_tf = tf.io.read_file(label_paths[0])
label = tf.io.decode_png(label_tf)

label_c, _ = tf.split(label, [1, -1], axis=-1)
label_c = tf.image.resize(label_c, input_shape[:2], 'nearest')
label = tf.squeeze(label_c)    
label_onehot = tf.one_hot(label, len(class_nums))

(128, 128, 1)


# Preprare Dataset

In [5]:
def data_preprocess(img_path, lbl_path):
    image_tf = tf.io.read_file(img_path)
    image = tf.io.decode_jpeg(image_tf) / 255
    image = tf.image.resize(image, input_shape[:2])
    
    label_tf = tf.io.read_file(lbl_path)
    label = tf.io.decode_png(label_tf)

    label_c, _ = tf.split(label, [1, -1], axis=-1)
    label_c = tf.image.resize(label_c, input_shape[:2], 'nearest')
    label = tf.squeeze(label_c)    
    label_onehot = tf.one_hot(label, len(class_nums))
    return image, label_onehot

In [6]:
dataset = tf.data.Dataset.from_tensor_slices((image_paths, label_paths))
dataset = dataset.map(data_preprocess)
dataset = dataset.batch(batch_size)
dataset = dataset.shuffle(10)
dataset = dataset.repeat()

# Modeling

In [7]:
# Build UNet
inputs = layers.Input(input_shape)
conv1 = layers.Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(inputs)
conv1 = layers.BatchNormalization()(conv1)
conv1 = layers.Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(conv1)
conv1 = layers.BatchNormalization()(conv1)
conv1 = layers.Activation("relu")(conv1)
pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = layers.Conv2D(128, 3, padding = 'same', kernel_initializer = 'he_normal')(pool1)
conv2 = layers.BatchNormalization()(conv2)
conv2 = layers.Activation("relu")(conv2)
conv2 = layers.Conv2D(128, 3, padding = 'same', kernel_initializer = 'he_normal')(conv2)
conv2 = layers.BatchNormalization()(conv2)
conv2 = layers.Activation("relu")(conv2)
pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = layers.Conv2D(256, 3, padding = 'same', kernel_initializer = 'he_normal')(pool2)
conv3 = layers.BatchNormalization()(conv3)
conv3 = layers.Activation("relu")(conv3)
conv3 = layers.Conv2D(256, 3, padding = 'same', kernel_initializer = 'he_normal')(conv3)
conv3 = layers.BatchNormalization()(conv3)
conv3 = layers.Activation("relu")(conv3)
pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = layers.Conv2D(512, 3, padding = 'same', kernel_initializer = 'he_normal')(pool3)
conv4 = layers.BatchNormalization()(conv4)
conv4 = layers.Activation("relu")(conv4)
conv4 = layers.Conv2D(512, 3, padding = 'same', kernel_initializer = 'he_normal')(conv4)
conv4 = layers.BatchNormalization()(conv4)
conv4 = layers.Activation("relu")(conv4)
drop4 = layers.Dropout(drop_rate)(conv4)
pool4 = layers.MaxPooling2D(pool_size=(2, 2))(drop4)

conv5 = layers.Conv2D(1024, 3, padding = 'same', kernel_initializer = 'he_normal')(pool4)
conv5 = layers.BatchNormalization()(conv5)
conv5 = layers.Activation("relu")(conv5)
conv5 = layers.Conv2D(1024, 3, padding = 'same', kernel_initializer = 'he_normal')(conv5)
conv5 = layers.BatchNormalization()(conv5)
conv5 = layers.Activation("relu")(conv5)
drop5 = layers.Dropout(drop_rate)(conv5)

up6 = layers.Conv2DTranspose(1024, 2, padding='same', strides=(2, 2))(drop5)
merge6 = layers.concatenate([drop4, up6], axis=3)
conv6 = layers.Conv2D(512, 3, padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = layers.BatchNormalization()(conv6)
conv6 = layers.Activation("relu")(conv6)
conv6 = layers.Conv2D(512, 3, padding = 'same', kernel_initializer = 'he_normal')(conv6)
conv6 = layers.BatchNormalization()(conv6)
conv6 = layers.Activation("relu")(conv6)

up7 = layers.Conv2DTranspose(512, 2, padding='same', strides=(2, 2))(conv6)
merge7 = layers.concatenate([conv3, up7], axis=3)
conv7 = layers.Conv2D(256, 3, padding = 'same', kernel_initializer = 'he_normal')(merge7)
conv7 = layers.BatchNormalization()(conv7)
conv7 = layers.Activation("relu")(conv7)
conv7 = layers.Conv2D(256, 3, padding = 'same', kernel_initializer = 'he_normal')(conv7)
conv7 = layers.BatchNormalization()(conv7)
conv7 = layers.Activation("relu")(conv7)

up8 = layers.Conv2DTranspose(256, 2, padding='same', strides=(2, 2))(conv7)
merge8 = layers.concatenate([conv2, up8], axis=3)
conv8 = layers.Conv2D(128, 3, padding = 'same', kernel_initializer = 'he_normal')(merge8)
conv8 = layers.BatchNormalization()(conv8)
conv8 = layers.Activation("relu")(conv8)
conv8 = layers.Conv2D(128, 3, padding = 'same', kernel_initializer = 'he_normal')(conv8)
conv8 = layers.BatchNormalization()(conv8)
conv8 = layers.Activation("relu")(conv8)

up9 = layers.Conv2DTranspose(128, 2, padding='same', strides=(2, 2))(conv8)
merge9 = layers.concatenate([conv1, up9], axis=3)
conv9 = layers.Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(merge9)
conv9 = layers.BatchNormalization()(conv9)
conv9 = layers.Activation("relu")(conv9)
conv9 = layers.Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv9 = layers.BatchNormalization()(conv9)
conv9 = layers.Activation("relu")(conv9)
conv9 = layers.Conv2D(2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
conv10 = layers.Conv2D(len(class_nums), 1, activation = 'sigmoid')(conv9)

model = tf.keras.Model(inputs=inputs, outputs=conv10)

# Optimization

In [8]:
# Dice Coefficient
def precision(y_true, y_pred):
    axes = tuple(range(1, len(y_pred.shape)-1))
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axes)
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)), axes)
    precision = true_positives / (predicted_positives + K.epsilon())
    
    return K.mean(precision)


def recall(y_true, y_pred):
    axes = tuple(range(1, len(y_pred.shape)-1))
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axes)
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)), axes)
    recall = true_positives / (possible_positives + K.epsilon())
    
    return K.mean(recall)

def dice(y_true, y_pred):
    epsilon=1e-6
    axes = tuple(range(1, len(y_pred.shape)-1)) 
    numerator = 2. * K.sum(y_pred * y_true, axes)
    denominator = K.sum(K.square(y_pred) + K.square(y_true), axes)
    
    return K.mean(numerator / (denominator + epsilon))

def soft_dice_loss(y_true, y_pred):
    epsilon=1e-6
    axes = tuple(range(1, len(y_pred.shape)-1)) 
    numerator = 2. * K.sum(y_pred * y_true, axes)
    denominator = K.sum(K.square(y_pred) + K.square(y_true), axes)
    
    return 1 - K.mean(numerator / (denominator + epsilon))

In [9]:
model.compile(loss=soft_dice_loss,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy', dice])

In [10]:
step_per_epochs = len(image_paths) // batch_size

model.fit(dataset, steps_per_epoch=step_per_epochs)

 5/16 [========>.....................] - ETA: 37s - loss: 0.9624 - accuracy: 0.0430 - dice: 0.0376

KeyboardInterrupt: 